In [1]:
import os
import pandas as pd
from pathlib import Path
import librosa
from IPython.display import HTML

from moviepy.editor import *
import moviepy.video.VideoClip
import moviepy.editor as mpy
from moviepy.editor import VideoFileClip, concatenate_videoclips

from tqdm import tqdm_notebook as tqdm

In [2]:
base_path = Path('../data/taylor') # change what comes after "data" for your computer location
input_audio_path = '../data/music-from-youtube/Arctic Monkeys - Do I Wanna Know (Official Video)-bpOSxM0rNPM/Arctic Monkeys - Do I Wanna Know (Official Video)-bpOSxM0rNPM.webm.mp3'
#'../data/playlist RICK AND MORTY SONGS/Seasons 1-3 Opening Credits _ Rick and Morty _ adult swim-DLaqu2QJYPY.mp3'

taylor_swifts = [path for path in base_path.glob('Taylor Swift*') if path.is_dir()] # create a list of taylor song folder paths
largest_files = [max(taylor_song.glob('*'), key=lambda f: f.stat().st_size) for taylor_song in taylor_swifts]

In [3]:
taylor_df = pd.read_pickle('taylor_df.pkl')
taylor_df.head(3)

,name,song_wave,scene_start_times_sec,onset_times_sec
0,Taylor Swift - White Horse-D1Xr-JFLxik,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0 0.000 1 0.667 2 3.587 3...,"[0.6965986394557823, 1.3467573696145125, 1.648..."
1,Taylor Swift - The Story Of Us-nN6VR92V70M,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0 0.000 1 0.500 2 1.417 3...,"[0.6501587301587302, 0.905578231292517, 1.0913..."
2,Taylor Swift - Delicate-tCXGJQYZ9JA,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0 0.000 1 3.045 2 6.715 3 ...,"[3.8312925170068026, 4.040272108843538, 4.2028..."


In [4]:
video_clips = dict()

for i in largest_files:
    v = mpy.VideoFileClip(str(i))
    video_clips.update({i.stem:v})

print('len video_clips = ', len(video_clips))

len video_clips =  30


In [5]:
song_cuts = pd.read_pickle('arcticmonkeys2.pkl')
print(song_cuts.shape)
song_cuts.head(3)

(53, 3)


,name,start,end
0,Taylor Swift - You Need To Calm Down-Dkk9gvTmCXY,85.294,87.337
1,Taylor Swift - You Need To Calm Down-Dkk9gvTmCXY,75.325,76.994
2,Taylor Swift - You Need To Calm Down-Dkk9gvTmCXY,76.994,77.744


In [6]:
final_cut= []

for index, row in song_cuts.iterrows():
    v_time = [row.start, row.end]
    v = video_clips[row['name']]

    final_cut.append(v.subclip(*v_time).resize(width=720))   
    
audio = mpy.AudioFileClip(str(input_audio_path))

In [7]:
joined = mpy.concatenate_videoclips(final_cut, method='compose')
joined.audio = audio#.subclip(*audio_times)

In [9]:
# joined.ipython_display(width=720)
joined.write_videofile("videoThis.mp4") # Will fail ! NO DURATION !

chunk:  72%|███████▏  | 4222/5860 [00:20<00:03, 454.52it/s, now=None]